In [19]:
from datetime import datetime
import os

import reflexio

client = reflexio.ReflexioClient()
client.api_key = client.login("peakmojo", "s").access_token


## Publish interaction

publish text interaction example

In [17]:
client.publish_interaction(
    request_id="your_request_id",  # for attribution for profile sources
    user_id="your_user_id",
    interaction_requests=[
        {
            'timestamp': 1738572096,
            'content': 'Hey this is Yi calling from kiku sushi, we offer salmon sushi, are you interested?',
            'role': 'Sales Rep'
        }
    ]
)

PublishUserInteractionResponse(success=True, message='')

### batch publish interactions

In [6]:
import pandas as pd
import re

def parse_dialogue(text):
    # Find the dialogue section
    dialogue_match = re.search(r'<DIALOG_EXAMPLE>(.*?)</DIALOG_EXAMPLE>', text, re.DOTALL)
    if not dialogue_match:
        return pd.DataFrame(columns=['Role', 'Content'])
    
    dialogue = dialogue_match.group(1).strip()
    
    # Split into conversation turns
    turns = dialogue.split('\n\n')
    
    # Process each turn
    conversation_data = []
    for turn in turns:
        if ':' in turn:
            # Split at first colon to separate role from content
            role, content = turn.split(':', 1)
            
            # Clean up the role and content
            role = role.strip()
            content = content.strip()
            
            # Map roles to consistent format
            role = 'Customer' if role == 'Customer' else 'Sales Rep'
            
            conversation_data.append({
                'Role': role,
                'Content': content
            })
    
    # Create DataFrame
    df = pd.DataFrame(conversation_data)
    return df

In [16]:
fp  = '../experiments/peakmojo/h2gjw2n6ud/ibrssunqfx_updated.txt'

def add_interaction_from_fp(fp):
    with open(fp, 'r') as f:
        raw_conversation = f.read()
        df = parse_dialogue(raw_conversation)
        interactions = list(df[df['Role'] == 'Sales Rep']['Content'])
        user_id = os.path.splitext(os.path.basename(fp))[0]
        print(f"processing user id: {user_id}")

    
        for i, interaction in enumerate(interactions):
            interaction_requests = [{
                'timestamp': int(datetime.utcnow().timestamp()),
                'content': interaction,
                'role': 'Sales Rep'
            }]
            
            client.publish_interaction(
            request_id=f"request_{user_id}_{i}",  # for attribution for profile sources
            user_id=user_id,
            interaction_requests=interaction_requests)
            print(interaction_requests)



In [18]:
add_interaction_from_fp(fp)

processing user id: ibrssunqfx_updated
[{'timestamp': 1738679975, 'text_interaction': "Hey, may I speak with Bob? Hello? Who's this?", 'role': 'Sales Rep'}]
[{'timestamp': 1738679976, 'text_interaction': "Hey, Bob, this is Kiara. I'm calling with Fundera. It looks like we helped you secure some financing with our partners at Reliant Funding back in May of 2020.", 'role': 'Sales Rep'}]
[{'timestamp': 1738679978, 'text_interaction': 'Yeah, I just wanted to reach out to you to see if you had any upcoming projects where some additional working capital could be helpful.', 'role': 'Sales Rep'}]
[{'timestamp': 1738679979, 'text_interaction': 'Yeah, for sure. You want to make some upgrades and extend your product line? We could definitely help with that. Do you happen to still have that loan with Reliant Funding?', 'role': 'Sales Rep'}]
[{'timestamp': 1738679981, 'text_interaction': 'We have a variety of options. We have short term loans, medium term loans, as well as the SBA 7A, which a lot o

## Delete operations

In [22]:
# delete interaction
client.delete_interaction(
    user_id="your_user_id",
    interaction_id="your_user_id_your_request_id_762bddb4"
)

DeleteUserInteractionResponse(success=True, message='')

In [21]:
# delete user profile
client.delete_profile(
    user_id="your_user_id",
    profile_id='dc4925f6-92b2-4907-9bff-d2c103df3050'
)

DeleteUserProfileResponse(success=True, message='')

## Get changelog, user profiles and interactions

In [ ]:
logs = client.get_profile_change_log()
logs.model_dump()

In [ ]:
interactions = client.get_interactions(
    request = {
        "user_id": "ibrssunqfx_updated",
        "top_k": 4
    }
)
interactions.model_dump()

In [ ]:
profiles = client.get_profiles(
    request = {
        "user_id": "ibrssunqfx_updated",
        "top_k": 4
    }
)
profiles.model_dump()